# Debug the real spectra

So it doesn't work well with real data, so having taken out one example of real data, the sequence is

__NOTE__: These are ALL hybrids


1. `DLQTLALLL`
2. `DPQVAQLELGGEVEDPQVAQLELGGGPGAG`
3. `DLPVNSPMTKG`
4. `DLQTLALWSRM`
5. `DLQTLALNAAR`
6. `DLQTLALNAAR`
7. `DLQTLEVE`
8. `DLQTLALEVE`

so toy with the tool until we find out whats wrong

In [1]:
correct = ['DLQTLALLL', 'DPQVAQLELGGEVEDPQVAQLELGGGPGAG', 'DLPVNSPMTKG', 'DLQTLALWSRM', 'DLQTLALNAAR', 'DLQTLALNAAR', 'DLQTLEVE', 'DLQTLALEVE']


## Load params and run

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src import runner
    

In [3]:
specPath = '/Users/zacharymcgrath/Desktop/nod2 data/hybridSpec/'
fastaPath = '/Users/zacharymcgrath/Desktop/nod2 data/filteredNOD2.fasta'
outputDir = '/Users/zacharymcgrath/Desktop/Experiment output/hybridReal/'
minPep = 3
maxPep = 30
tolerance = 20
relative_abundance_filter = 0.005
precursor_tolerance = 1
peak_filter = 0
verbose = True
scoringAlg = 'ion'

In [4]:
params = {
    'spectra_folder': specPath,
    'database_file': fastaPath,
    'output_dir': outputDir,
    'min_peptide_len': minPep,
    'max_peptide_len': maxPep,
    'tolerance': tolerance,
    'precursor_tolerance': precursor_tolerance,
    'peak_filter': peak_filter, 
    'relative_abundance_filter': relative_abundance_filter,
    'verbose': verbose, 
    'scoring_alg': scoringAlg, 
    'DEBUG': True
}

In [5]:
%%time
runner.run(params)

Loading database...
Done
Loading spectra...
Done
Making the protein mass set...
On protein 280/280 [100%]
Sorting the set of protein masses...
Done. Length of the list: 6390463
Time to build: 38.0003 seconds
Time to do merge: 1.5198 seconds
Creating an alignment for 8/8 [100%]
Finished search. Writting results to /Users/zacharymcgrath/Desktop/Experiment output/hybridReal/...
Could not make an alignment for 0/8 spectra (0%)
CPU times: user 1min 5s, sys: 1.03 s, total: 1min 6s
Wall time: 1min 6s


## Load and read results

In [6]:
import pandas as pd

outputFile = outputDir + 'summary.tsv'
hybridOutputFile = outputDir + 'hybrid_summary.tsv'

In [7]:
df = pd.read_csv(outputFile, sep='\t')
hdf = pd.read_csv(hybridOutputFile, sep='\t')

In [8]:
hdf.head(10)

,Unnamed: 0,left_proteins,right_proteins,sequence,hybrid_sequence,b_score,y_score,total_score,precursor_distance,entry name,id
0,0,['sp|Q91ZI0|CELR3_MOUSE Cadherin EGF LAG seven...,['sp|Q8BV79|TRNK1_MOUSE TPR and ankyrin repeat...,TLAITAPSII,TLAITA-PSII,5,3,8,0.000194,0,NOD2_E3.21510.21510.2.pkl
1,1,['sp|P01326|INS2_MOUSE Insulin-2 OS=Mus muscul...,['sp|P97412|LYST_MOUSE Lysosomal-trafficking r...,DPQVAQLELGGGPVGHVHCREIICSVAF,DPQVAQLELGGGP-VGHVHCREIICSVAF,12,4,16,0.000308,1,NOD2_E3.12771.12902.3.pkl
2,2,['sp|P01325|INS1_MOUSE Insulin-1 OS=Mus muscul...,['sp|P26339|CMGA_MOUSE Chromogranin-A OS=Mus m...,DLQTLALWSRM,DLQTLAL-WSRM,5,8,13,0.000192,3,NOD2_E3.16373.16401.2.pkl
3,3,['sp|P01325|INS1_MOUSE Insulin-1 OS=Mus muscul...,['sp|NOD0001|IAPP_MOUSE NOD Islet amyloid poly...,DLQTLALNAAR,DLQTLAL-NAAR,4,9,13,0.000812,4,NOD2_E3.10614.10681.2.pkl
4,4,['sp|P01325|INS1_MOUSE Insulin-1 OS=Mus muscul...,['sp|NOD0001|IAPP_MOUSE NOD Islet amyloid poly...,DLQTLALNAAR,DLQTLAL-NAAR,4,10,14,0.504749,5,NOD2_E3.10635.10674.3.pkl
5,5,['sp|P01027|CO3_MOUSE Complement C3 OS=Mus mus...,"['sp|Q9CR21|ACPM_MOUSE Acyl carrier protein, m...",ELTNIEVE,ELTNIE-VE,6,2,8,0.000293,6,NOD2_E3.11427.11516.2.pkl


In [9]:
df.head(10)

,Unnamed: 0,proteins,sequence,b_score,y_score,total_score,precursor_distance,entry name,id
0,0,[],DIPVNSPMTKG,2,5,7,0.000452,2,NOD2_E3.7065.7065.2.pkl
1,1,[],DLQTLALEVE,7,4,11,0.004477,7,NOD2_E3.16920.17393.2.pkl


In [10]:
import json

s = json.load(open(outputDir + 'summary.json', 'r'))

In [11]:
for i, (k, v) in enumerate(s.items()):
    c = correct[i]
    print(f'Alignment for spectrum {i} \t "correct": {c} \n ==============================')
    for a in v['alignments']:
        print(f'{a["sequence"]}: \t {a["total_score"]}, \t {a["b_score"]}, \t {a["y_score"]}')
    print()

    

Alignment for spectrum 0 	 "correct": DLQTLALLL 
TLAITAPSII: 	 8, 	 5, 	 3
DLQTLALIL: 	 8, 	 6, 	 2
DLQTLALII: 	 8, 	 6, 	 2
ITALATSPLL: 	 8, 	 5, 	 3
TLAITASPIL: 	 8, 	 5, 	 3

Alignment for spectrum 1 	 "correct": DPQVAQLELGGEVEDPQVAQLELGGGPGAG 
DPQVAQLELGGGPVGHVHCREIICSVAF: 	 16, 	 12, 	 4
PGAGDLQTLALEVAEDPQVAQLELGGGPGAG: 	 16, 	 7, 	 9
PGAGDLQTLALEVAQDPQVAQLELGGGPGAG: 	 16, 	 7, 	 9
DPQVAQLELGGSEESALNHLQGAGGAEPR: 	 14, 	 9, 	 5
APGGSNVVAVTAVDVEDPQVAQLELGGGPGAG: 	 13, 	 5, 	 8

Alignment for spectrum 2 	 "correct": DLPVNSPMTKG 
DIPVNSPMTKG: 	 7, 	 2, 	 5
LDPVNSPMTKG: 	 7, 	 2, 	 5
EVPVNSPMTKG: 	 7, 	 2, 	 5
IDPVNSPMTKG: 	 7, 	 2, 	 5
VEPVNSPMTKG: 	 7, 	 2, 	 5

Alignment for spectrum 3 	 "correct": DLQTLALWSRM 
DLQTLALWSRM: 	 13, 	 5, 	 8
DLQTLALEFGLGG: 	 12, 	 5, 	 7
DLQTLAITFTNP: 	 12, 	 5, 	 7
DLQTLANSESKK: 	 11, 	 5, 	 6
DLQTLALGQIFD: 	 11, 	 5, 	 6

Alignment for spectrum 4 	 "correct": DLQTLALNAAR 
DLQTLALNAAR: 	 13, 	 4, 	 9
DLQTLALQQR: 	 12, 	 4, 	 8
DLQTLAIQQR: 	 12, 	 4, 	